# **Train your YOLOv8 keypoint model.**

First, fill in your **apikey** and **project name** below. Next, start the training script by clicking "Runtime -> Run all"

For more advanced options like setting the number of epochs or changing the base model, look in the documentation.

Once the training is finished, the new model is added to the trainyolo platform under the project models tab. There you can compare scores with other versions, and use this model for model-assisted labeling.

In [2]:
# DATASET PARAMETERS
API_KEY = "dcf21adf6ff11f6a6440a363e965d269f6ccc08a"
PROJECT_NAME = "Sitting pose"

# YOLO PARAMETERS
IM_SIZE = 640
N_EPOCHS = 50 # you can lower this if you have a large dataset
BATCH_SIZE = 4 # you can increase this if you have a large dataset, > 50 images, you can increase this to 4, >100 images you can increase this to 8
MODEL = 'yolov8n-pose.pt'

In [2]:
# install yolov8
%pip install -qU ultralytics

# install trainyolo sdk
%pip install -qU trainyolo-py

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\BAPS\AppData\Local\Temp\pip-install-58rlph63\trainyolo-py_6ac180cd4b174c4c8262ba2842b095cf\setup.py", line 5, in <module>
      from Cython.Build import cythonize
  ModuleNotFoundError: No module named 'Cython'
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
from trainyolo.client import Client, Project

# init client
client = Client(API_KEY)

# load project
project = Project.get_by_name(client, PROJECT_NAME)
project_location = project.pull(location='C:/Users/BAPS/Documents/Pose_estimation/good_bad_sitting_pose/data', format='yolov8')

ModuleNotFoundError: No module named 'trainyolo'

In [ ]:
# Train yolo model
!yolo task=pose mode=train model=$MODEL data="{project_location}/dataset.yaml" epochs=$N_EPOCHS imgsz=$IM_SIZE batch=$BATCH_SIZE

Ultralytics YOLOv8.0.171 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=C:/Users/BAPS/Documents/Pose_estimation/good_bad_sitting_pose/data/Sitting pose/dataset.yaml, epochs=50, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, re

In [ ]:
# upload model
from trainyolo.utils.yolov8 import upload_yolov8_run

upload_yolov8_run(project, mode='pose', iou=0.45)

Reading best conf from F1_curve
Unable to find f1 curve, defaulting to conf=0.5
adding weights: ./runs/pose/train3/weights/best.pt to project ...


In [1]:
import cv2
import numpy as np

# Load the YOLOv8 model
model = cv2.dnn.readNet('yolov8.weights', 'yolov8.cfg')

# Load the video
cap = cv2.VideoCapture('video.mp4')

# Get the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create an output video
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (frame_width, frame_height))

while True:
    # Read the next frame
    ret, frame = cap.read()

    # Convert the frame to a blob
    blob = cv2.dnn.blobFromImage(frame, 1.0, (frame_width, frame_height), (104, 117, 123), swapRB=True, crop=False)

    # Pass the blob through the network
    model.setInput(blob)
    detections = model.forward()

    # Loop over the detections
    for i in range(0, detections.shape[2]):
        # Get the confidence score
        confidence = detections[0, 0, i, 2]

        # If the confidence is greater than a threshold, draw the bounding box and the keypoints
        if confidence > 0.5:
            x = int(detections[0, 0, i, 3] * frame_width)
            y = int(detections[0, 0, i, 4] * frame_height)
            w = int(detections[0, 0, i, 5] * frame_width)
            h = int(detections[0, 0, i, 6] * frame_height)

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Draw the keypoints
            for j in range(0, 17):
                cx = int(detections[0, 0, i, 13 + 2 * j] * frame_width) + x
                cy = int(detections[0, 0, i, 13 + 2 * j + 1] * frame_height) + y
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Video', frame)

    # Check if the user wants to quit
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov8.cfg in function 'cv::dnn::dnn4_v20230620::readNetFromDarknet'
